## Analytical context

This case is based on a challenge proposed by Red Hat on the Kaggle platform.

Red Hat is able to obtain a significant amount of behavioral information from its customers. The interest of the company is based on the search for better methods to analyze its customers to predict on which they should make a special emphasis that could become high value customers.

This challenge aims to create a ranking algorithm that identifies customers with the greatest potential to become high-value customers for Red Hat based on their activity.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/My Drive/Colab Notebooks/Neuronal-Networks-with-Keras-Old/'
%ls

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/Neuronal-Networks-with-Keras-Old
'11.-Clasificación Binaria.ipynb'
'3.-Estructura de redes neuronales.ipynb'
'4.-Creando nuestra primer red neuronal.ipynb'
 datasets/
'Design, Training and Evaluation of the Model.ipynb'
'Feature Engineering.ipynb'


In [ ]:
# Import the libraries

import pandas as pd
import numpy as np

people = pd.read_csv('datasets/people.zip', sep=',')
people.head(3)

,people_id,char_1,group_1,char_2,date,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,char_11,char_12,char_13,char_14,char_15,char_16,char_17,char_18,char_19,char_20,char_21,char_22,char_23,char_24,char_25,char_26,char_27,char_28,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,ppl_100,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,type 2,type 2,True,False,False,True,True,False,True,False,False,False,False,True,False,False,False,False,False,True,True,False,True,True,False,False,True,True,True,False,36
1,ppl_100002,type 2,group 8688,type 3,2021-01-06,type 28,type 9,type 5,type 3,type 11,type 2,type 4,False,False,True,True,False,False,False,True,False,False,False,False,False,True,False,True,True,True,False,False,True,True,True,True,True,True,True,False,76
2,ppl_100003,type 2,group 33592,type 3,2022-06-10,type 4,type 8,type 5,type 2,type 5,type 2,type 2,True,True,True,True,True,True,False,True,False,True,False,True,True,True,True,True,True,True,True,False,False,True,True,True,True,False,True,True,99


The first fact to highlight is that the information is presented anonymized.

A large number of fields of a categorical nature are appreciated.

In [ ]:
activity = pd.read_csv('datasets/act_train.zip', sep=',')

activity.head(3)

,people_id,activity_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,outcome
0,ppl_100,act2_1734928,2023-08-26,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 76,0
1,ppl_100,act2_2434093,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
2,ppl_100,act2_3404049,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0


You can see that there is the same key in the people_id column, so you can start thinking about some kind of cross between these data.

# <h1 id="ingenieria">Feature Engineering</h1>

We will now evaluate the size of the datasets and the completeness of the information in the different available variables.

### Delete null data

In [ ]:
# Show the shape

print(people.shape)

# Show the null percent

100*people.isnull().sum()/people.shape[0]

(189118, 41)


people_id    0.0
char_1       0.0
group_1      0.0
char_2       0.0
date         0.0
char_3       0.0
char_4       0.0
char_5       0.0
char_6       0.0
char_7       0.0
char_8       0.0
char_9       0.0
char_10      0.0
char_11      0.0
char_12      0.0
char_13      0.0
char_14      0.0
char_15      0.0
char_16      0.0
char_17      0.0
char_18      0.0
char_19      0.0
char_20      0.0
char_21      0.0
char_22      0.0
char_23      0.0
char_24      0.0
char_25      0.0
char_26      0.0
char_27      0.0
char_28      0.0
char_29      0.0
char_30      0.0
char_31      0.0
char_32      0.0
char_33      0.0
char_34      0.0
char_35      0.0
char_36      0.0
char_37      0.0
char_38      0.0
dtype: float64

In [ ]:
# We repet with the other dataset 

print(activity.shape)
100*activity.isnull().sum()/activity.shape[0]

# We gonna delete the columns 90% null and fill

(2197291, 15)


people_id             0.000000
activity_id           0.000000
date                  0.000000
activity_category     0.000000
char_1               92.826849
char_2               92.826849
char_3               92.826849
char_4               92.826849
char_5               92.826849
char_6               92.826849
char_7               92.826849
char_8               92.826849
char_9               92.826849
char_10               7.173151
outcome               0.000000
dtype: float64

It is evident that the fields char_1 to char_9 have 92% of their values in null. This amount is very high to apply imputation methods, so we will choose to eliminate the columns with this problem.

In the case of the categorical variable char_10 there is a proportion of nulls (7.17 \%) that we can replace by the mode.

In [ ]:
activity.columns

Index(['people_id', 'activity_id', 'date', 'activity_category', 'char_1',
       'char_2', 'char_3', 'char_4', 'char_5', 'char_6', 'char_7', 'char_8',
       'char_9', 'char_10', 'outcome'],
      dtype='object')

In [ ]:
# Delete columns with null values
activity.drop(columns=['char_1','char_2', 'char_3', 'char_4', 'char_5', 'char_6',
                       'char_7', 'char_8','char_9'],inplace=True)

# Fill char_10 with the mode
activity['char_10'].fillna(activity['char_10'].mode()[0], inplace=True)

# We data is clean of null dates
100*activity.isnull().sum()/activity.shape[0]

people_id            0.0
activity_id          0.0
date                 0.0
activity_category    0.0
char_10              0.0
outcome              0.0
dtype: float64

## Preparing for the merge

It is appreciated that there are two variables that have the same names in both data sets (date and char_10) so it is good practice to rename these columns to avoid confusion when unifying the sources.

In [ ]:
# Rename the columns 
activity = activity.rename(columns={"date":"fecha_actividad","char_10":"tipo_actividad"})

In [ ]:
# Verify the change
print(activity.columns)
print(people.columns)

Index(['people_id', 'activity_id', 'fecha_actividad', 'activity_category',
       'tipo_actividad', 'outcome'],
      dtype='object')
Index(['people_id', 'char_1', 'group_1', 'char_2', 'date', 'char_3', 'char_4',
       'char_5', 'char_6', 'char_7', 'char_8', 'char_9', 'char_10', 'char_11',
       'char_12', 'char_13', 'char_14', 'char_15', 'char_16', 'char_17',
       'char_18', 'char_19', 'char_20', 'char_21', 'char_22', 'char_23',
       'char_24', 'char_25', 'char_26', 'char_27', 'char_28', 'char_29',
       'char_30', 'char_31', 'char_32', 'char_33', 'char_34', 'char_35',
       'char_36', 'char_37', 'char_38'],
      dtype='object')


Now we unify the datasets using the merge function and evaluate the consistency after this operation

In [ ]:
consolidado = activity.merge(people,on=["people_id"],how="inner")
print("Activity previous size: ",activity.shape)
print("People previous size: ",people.shape)
print()
print("New size: ",consolidado.shape)

Activity previous size:  (2197291, 6)
People previous size:  (189118, 41)

New size:  (2197291, 46)


# Data transformation

The objective variable that classifies the activity of users as potential clients is called outcome, we will evaluate the distribution of this variable:

In [ ]:
print("Outcome distribution: \n", 100*consolidado["outcome"].value_counts()/consolidado.shape[0])

Outcome distribution: 
 0    55.60456
1    44.39544
Name: outcome, dtype: float64


As we have mentioned, neural networks require input only numerical values, so we must convert those that are not. The consolidated dataset has 46 Boolean, numeric or categorical variables that must be transformed.

In [ ]:
types = pd.DataFrame(consolidado.dtypes)
print("Types of variables: ",types.groupby(0).size())

Types of variables:  0
bool      28
int64      2
object    16
dtype: int64


Boolean variables to integer variables

In [ ]:
# replace 'False' values by 0, and 'True' by 1
consolidado = consolidado.replace({False: 0, True: 1})

# recount of variable types
types = pd.DataFrame(consolidado.dtypes)
print("Types of variables after replacement: ",types.groupby(0).size())


Types of variables after replacement:  0
int64     30
object    16
dtype: int64


Variables object (string) to integer variables If we remember, the identifier people_id consists of a prefix "ppl_" followed by a unique number per user. In this case, it is enough to cut the prefix to transform this variable into a numeric one.

In [ ]:
consolidado.people_id = consolidado.people_id.str.slice(start=4).astype(float).astype(int)

types = pd.DataFrame(consolidado.dtypes)
print("Types of variables after 2nd replacement: ",types.groupby(0).size())

Types of variables after 2nd replacement:  0
int64     31
object    15
dtype: int64


When validating the consolidated dataset we can apply the same methodology previously mentioned to another pair of columns that have the same characteristics:

* activity_id,
* activity_category,
* group_1,
* tipo_actividad.

In [ ]:
consolidado.head(3)

,people_id,activity_id,fecha_actividad,activity_category,tipo_actividad,outcome,char_1,group_1,char_2,date,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,char_11,char_12,char_13,char_14,char_15,char_16,char_17,char_18,char_19,char_20,char_21,char_22,char_23,char_24,char_25,char_26,char_27,char_28,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,100,act2_1734928,2023-08-26,type 4,type 76,0,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,type 2,type 2,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,1,1,0,0,1,1,1,0,36
1,100,act2_2434093,2022-09-27,type 2,type 1,0,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,type 2,type 2,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,1,1,0,0,1,1,1,0,36
2,100,act2_3404049,2022-09-27,type 2,type 1,0,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,type 2,type 2,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,1,1,0,0,1,1,1,0,36


In [ ]:
consolidado[["activity_id", "activity_category", "group_1", "tipo_actividad"]].head(6)

,activity_id,activity_category,group_1,tipo_actividad
0,act2_1734928,type 4,group 17304,type 76
1,act2_2434093,type 2,group 17304,type 1
2,act2_3404049,type 2,group 17304,type 1
3,act2_3651215,type 2,group 17304,type 1
4,act2_4109017,type 2,group 17304,type 1
5,act2_898576,type 4,group 17304,type 1727


In [ ]:
consolidado.activity_id = consolidado.activity_id.str.slice(start=5).astype(float).astype(int)
consolidado.activity_category = consolidado.activity_category.str.slice(start=5).astype(float).astype(int)
consolidado.group_1 = consolidado.group_1.str.slice(start=6).astype(float).astype(int)
consolidado.tipo_actividad = consolidado.tipo_actividad.str.slice(start=5).astype(float).astype(int)

types = pd.DataFrame(consolidado.dtypes)
print("Variable types after 3rd replacement: ",types.groupby(0).size())

Variable types after 3rd replacement:  0
int64     35
object    11
dtype: int64


Let's evaluate the variables that follow object type and the number of different values they have

In [ ]:
types[types.values == 'object'].index

Index(['fecha_actividad', 'char_1', 'char_2', 'date', 'char_3', 'char_4',
       'char_5', 'char_6', 'char_7', 'char_8', 'char_9'],
      dtype='object')

In [ ]:
# variables de tipo object
objects_list = list(types[types.values == 'object'].index)
objects_list

['fecha_actividad',
 'char_1',
 'char_2',
 'date',
 'char_3',
 'char_4',
 'char_5',
 'char_6',
 'char_7',
 'char_8',
 'char_9']

In [ ]:
# number of different securities they own
consolidado[objects_list].nunique()

fecha_actividad     411
char_1                2
char_2                3
date               1196
char_3               43
char_4               25
char_5                9
char_6                7
char_7               25
char_8                8
char_9                9
dtype: int64

# Date type variables

From the previous list we highlight two date type variables (date, activity_date). For the neural network it represents an opportunity to be able to extract a series of additional characteristics that allow it to understand seasonal patterns of different periodicities (daily, weekly, monthly, etc.) that is why we will create a series of additional variables (of a numerical nature) from these two dates:

In [ ]:
#We convert the object variable into datetime
consolidado["date"] = pd.to_datetime(consolidado["date"])
#Creamos nuevas variables
consolidado["dia"] = consolidado["date"].dt.day
consolidado["dia_semana"] = consolidado["date"].dt.weekday
consolidado["semana"] = consolidado["date"].dt.week
consolidado["mes"] = consolidado["date"].dt.month
consolidado["trimestre"] = consolidado["date"].dt.quarter
consolidado["año"] = consolidado["date"].dt.year

# We repeat the previous procedure with activity_date
consolidado["fecha_actividad"] = pd.to_datetime(consolidado["fecha_actividad"])
consolidado["dia_actividad"] = consolidado["fecha_actividad"].dt.day
consolidado["dia_semana_actividad"] = consolidado["fecha_actividad"].dt.weekday
consolidado["semana_actividad"] = consolidado["fecha_actividad"].dt.week
consolidado["mes_actividad"] = consolidado["fecha_actividad"].dt.month
consolidado["trimestre_actividad"] = consolidado["fecha_actividad"].dt.quarter
consolidado["año_actividad"] = consolidado["fecha_actividad"].dt.year

# We eliminate the original columns
del(consolidado["date"])
del(consolidado["fecha_actividad"])

types = pd.DataFrame(consolidado.dtypes)
print("Tipos de variables luego de 4to reemplazo",types.groupby(0).size())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  from ipykernel import kernelapp as app


Tipos de variables luego de 4to reemplazo 0
int64     47
object     9
dtype: int64


# One hot encoding

With the relative small amount of categorical variables that we handle, we are going to proceed to carry out the conversion by One Hot Encoding

In [ ]:
mi_consolidado = consolidado.copy(deep=True)
print(mi_consolidado.shape)

(2197291, 56)


In [ ]:
#A function is defined that captures the dataframe and the column to return a dataframe after OHE
def OneHotEncoding_df(df, columna):
  OHE_df = pd.get_dummies(columna+'_'+df[columna])
  return OHE_df

In [ ]:

#The columns to which OHE is to be performed is obtained 
objects_list = list(types[types.values == 'object'].index)
objects_list

['char_1',
 'char_2',
 'char_3',
 'char_4',
 'char_5',
 'char_6',
 'char_7',
 'char_8',
 'char_9']

In [ ]:
# OHE is performed for each category
for category in objects_list:
  mask = OneHotEncoding_df(mi_consolidado, category)
  print(f'column {category} transformed!')
  mi_consolidado.drop(category, axis=1, inplace=True)
  mi_consolidado = pd.concat([mi_consolidado, mask], axis=1)

print(f"Final size of the transformed dataset: {mi_consolidado.shape}")

column char_1 transformed!
column char_2 transformed!
column char_3 transformed!
column char_4 transformed!
column char_5 transformed!
column char_6 transformed!
column char_7 transformed!
column char_8 transformed!
column char_9 transformed!
Final size of the transformed dataset: (2197291, 178)


In [ ]:
mi_consolidado['outcome']

0          0
1          0
2          0
3          0
4          0
          ..
2197286    1
2197287    1
2197288    1
2197289    1
2197290    1
Name: outcome, Length: 2197291, dtype: int64

# Export dataframe

Since there is a dataset with a large amount of data (2,197,291 samples and 178 variables), the extent to which the data will be exported will be parquet, since it has a greater advantage than a CSV file

In [ ]:
mi_consolidado.to_parquet('./datasets/variables_consolidadas.parquet')